In [1]:
import numpy as np
import pandas as pd
import torch
import os
import joblib
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from collections import Counter

# 引入配置和模型
import config 
# 确保 models 里的 LSTMFeatureExtractor 是最新的（带 fc_final 的版本）
from models import LSTMFeatureExtractor 
from utility_uad_svm import load_data, create_sequences_for_svm, get_features

--- [Config] 正在使用的设备: cpu ---


In [2]:
def clean_features(F_feat):
    """清除特征中的 NaN 和 Inf 值。"""
    nan_count = np.sum(np.isnan(F_feat))
    inf_count = np.sum(np.isinf(F_feat))
    if nan_count > 0 or inf_count > 0:
        print(f"【⚠️ 清理】发现 {nan_count} NaN / {inf_count} Inf。正在修复...")
        F_feat = np.nan_to_num(F_feat, nan=0.0, posinf=1e8, neginf=-1e8) 
    return F_feat

In [3]:
def complex_balance_data(X_feat, y_labels, sat_types, random_state=42):
    """
    【关键修改】基于 标签(y) 和 卫星类型(sat_type) 进行四象限平衡。
    确保 GNSS/PL 的 好/坏 信号在 SVM 训练集中比例一致。
    """
    print(f"  - [平衡前] 总体分布: {Counter(y_labels)}")
    
    # 构建临时 DataFrame 方便索引
    df_temp = pd.DataFrame({
        'idx': range(len(X_feat)), 
        'y': y_labels, 
        'sat': sat_types
    })
    
    # 1. 划分四类人群
    # 假设 sat_type: 0=GNSS, 1=PL; y: 0=LOS(好), 1=Multipath(坏)
    idx_gnss_good = df_temp[(df_temp['sat'] == 0) & (df_temp['y'] == 0)]['idx'].values
    idx_gnss_bad  = df_temp[(df_temp['sat'] == 0) & (df_temp['y'] == 1)]['idx'].values
    idx_pl_good   = df_temp[(df_temp['sat'] == 1) & (df_temp['y'] == 0)]['idx'].values
    idx_pl_bad    = df_temp[(df_temp['sat'] == 1) & (df_temp['y'] == 1)]['idx'].values
    
    print(f"  - 组别统计: GNSS好:{len(idx_gnss_good)}, GNSS坏:{len(idx_gnss_bad)}, PL好:{len(idx_pl_good)}, PL坏:{len(idx_pl_bad)}")
    
    # 2. 找到短板（最小数量）
    # 注意：如果某一组是 0，需要特殊处理
    counts = [len(idx_gnss_good), len(idx_gnss_bad), len(idx_pl_good), len(idx_pl_bad)]
    counts = [c for c in counts if c > 0] # 过滤掉 0
    if not counts:
        return np.empty((0, X_feat.shape[1])), np.empty((0,))
        
    min_size = min(counts)
    print(f"  - 平衡基准数 (min_size): {min_size}")
    
    
    # 3. 欠采样函数
    def sample_indices(indices, n):
        if len(indices) == 0: return indices
        np.random.seed(random_state)
        return np.random.choice(indices, n, replace=False)

    # 4. 执行采样
    final_indices = np.concatenate([
        sample_indices(idx_gnss_good, min_size),
        sample_indices(idx_gnss_bad, min_size),
        sample_indices(idx_pl_good, min_size),
        sample_indices(idx_pl_bad, min_size)
    ])
    
    np.random.shuffle(final_indices)
    
    return X_feat[final_indices], y_labels[final_indices]

In [4]:
# def standardize_features_global(F_src, F_tgt):
#     """
#     对特征进行标准化。
#     逻辑：在源域上 Fit，应用到源域和目标域。
#     """
#     print("  - 执行特征标准化 (Z-Score)...")
#     scaler = StandardScaler()
#     scaler.fit(F_src) # 只在源域上学习均值和方差
    
#     # 防御标准差为 0
#     zero_std_mask = scaler.scale_ == 0
#     if np.any(zero_std_mask):
#         scaler.scale_[zero_std_mask] = 1e-8
        
#     F_src_scaled = scaler.transform(F_src)
#     F_tgt_scaled = scaler.transform(F_tgt)
#     return F_src_scaled, F_tgt_scaled

def standardize_features_global(F_src, F_tgt):
    print("  - 执行特征标准化 (Z-Score)...")
    scaler = StandardScaler()
    scaler.fit(F_src) # 计算源域的均值和方差
    
    # --- 🔍 强力调试与修复 ---
    print(f"  - [调试] 特征均值前5位: {scaler.mean_[:5]}")
    print(f"  - [调试] 特征方差前5位: {scaler.var_[:5]}")
    
    # 检查方差是否极小 (接近0)
    # 注意：StandardScaler 内部用 var_, 如果 var_ 为 0，scale_ 也会出问题
    zero_var_mask = scaler.var_ < 1e-8 # 使用一个极小阈值
    if np.any(zero_var_mask):
        num_dead = np.sum(zero_var_mask)
        print(f"【⚠️ 严重警告】发现 {num_dead} 个特征维度方差几乎为 0 (死神经元)！")
        # 强制把这些维度的方差设为 1，防止除以 0 (这样 x-mean 后结果就是 0，不会是 NaN)
        scaler.var_[zero_var_mask] = 1.0 
        scaler.scale_[zero_var_mask] = 1.0 
    
    F_src_scaled = scaler.transform(F_src)
    F_tgt_scaled = scaler.transform(F_tgt)
    
    return F_src_scaled, F_tgt_scaled

In [5]:
SRC_IDS = config.SRC_IDS
TGT_ID = config.TGT_ID
CSV_PATHS = config.CSV_PATHS

In [6]:
# =============================================================================
# 主程序
# =============================================================================

def main_single_svm():
    device = torch.device(config.DEVICE)
    print(f"--- [SVM Main] 正在使用设备: {device} ---")
    SRC_IDS = config.SRC_IDS
    TGT_ID = config.TGT_ID
    CSV_PATHS = config.CSV_PATHS
    # -------------------------------------------------------
    # 1. 加载数据 & 原始数据标准化 (必须与 DANN 训练时一致!)
    # -------------------------------------------------------
    print("\n--- [步骤 1] 加载原始数据 ---")
    df_src, df_tgt = load_data(SRC_IDS, TGT_ID, CSV_PATHS)
    
    # # 【重要提示】如果 DANN 训练时对原始数据做了标准化，这里必须做。
    # # 如果 DANN 训练时没做，请注释掉下面这几行！
    # print("  - [提示] 正在对原始输入信号进行标准化 (假设 DANN 训练时也做了)...")
    # raw_scaler = StandardScaler()
    # # fit on source
    # df_src[config.FEATURES] = raw_scaler.fit_transform(df_src[config.FEATURES])
    # # transform target
    # df_tgt[config.FEATURES] = raw_scaler.transform(df_tgt[config.FEATURES])
    # print("✅ 原始数据预处理完成。")

    # -------------------------------------------------------
    # 2. 制作序列 (需要获取 S_src 用于平衡)
    # -------------------------------------------------------
    print("\n--- [步骤 2] 制作序列 ---")
    # 注意：create_sequences_for_svm 需要返回 S_src (卫星类型)
    X_src, y_src, S_src, I_src_raw, X_tgt, y_tgt, S_tgt, TGT_INDICES = create_sequences_for_svm(df_src, df_tgt, config)
    print("--- [数据分布检查] ---")
    from collections import Counter
    print(f"目标域卫星分布: {Counter(S_tgt)}")
    if X_src is None: return

    # -------------------------------------------------------
    # 3. 加载特征提取器 (G_f)
    # -------------------------------------------------------
    print("\n--- [步骤 3] 加载 G_f 模型并提取特征 ---")
    # ！！请确保这里的参数与 models.py 中的定义一致！！
    # 如果 models.py 里加了 final_feature_dim，这里也要确认默认值是否匹配
    G_f = LSTMFeatureExtractor(
        input_size=config.LSTM_INPUT_SIZE,
        hidden_size=config.LSTM_HIDDEN_SIZE,
        num_layers=config.LSTM_NUM_LAYERS,
        dropout=config.LSTM_DROPOUT
    ).to(device)

    G_f_path = os.path.join(config.MODEL_SAVE_DIR, "G_f_final.pth")
    if not os.path.exists(G_f_path):
        print(f"❌ 错误：找不到模型文件 {G_f_path}")
        return
        
    G_f.load_state_dict(torch.load(G_f_path, map_location=device))
    G_f.eval() # 必须是 eval 模式！

    # 提取特征 (Raw Features)
    F_src_raw = get_features(G_f, X_src, config.BATCH_SIZE, device)
    if np.isnan(F_src_raw).any():
        print("【💥 爆炸】LSTM 提取出的原始特征里就已经有 NaN 了！模型训练废了！")
    else:
        print("【✅ 正常】原始特征里没有 NaN。问题出在标准化步骤。")
    F_tgt_raw = get_features(G_f, X_tgt, config.BATCH_SIZE, device)
    
    print(f"  - 源域特征形状: {F_src_raw.shape}")

    # -------------------------------------------------------
    # 4. 特征标准化 (Feature Standardization)
    # -------------------------------------------------------
    print("\n--- [步骤 4] 特征标准化 ---")
    # 先标准化，再平衡。这样Scaler是基于全量数据的，更准。
    F_src_scaled, F_tgt_scaled = standardize_features_global(F_src_raw, F_tgt_raw)
    
    # 清洗 (NaN/Inf)
    F_src_scaled = clean_features(F_src_scaled)
    F_tgt_scaled = clean_features(F_tgt_scaled)

    # -------------------------------------------------------
    # 5. 数据平衡 (四象限平衡) & 训练 SVM
    # -------------------------------------------------------
    print("\n--- [步骤 5] 数据平衡与 SVM 训练 ---")
    
    # 使用 Complex Balance (基于 y 和 S)
    F_bal, y_bal = complex_balance_data(F_src_scaled, y_src, S_src)
    
    # 二次采样 (如果数据量太大，SVM 跑不动)
    MAX_SVM_SAMPLES = 20000
    if len(F_bal) > MAX_SVM_SAMPLES:
        print(f"  - 数据量 {len(F_bal)} > {MAX_SVM_SAMPLES}，进行二次随机采样...")
        # 这里只需要随机采样即可，因为 F_bal 已经是比例平衡的了
        indices = np.random.choice(len(F_bal), MAX_SVM_SAMPLES, replace=False)
        F_bal = F_bal[indices]
        y_bal = y_bal[indices]

    print(f"  - 最终 SVM 训练集大小: {F_bal.shape}")

    # 训练
    print("  - 开始训练 SVM (RBF Kernel)...")
    svm_global = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42) 
    # C=1.0 通常比 0.5 更通用，gamma='scale' 自动处理特征维度
    svm_global.fit(F_bal, y_bal)
    print("✅ SVM 训练完成。")

    # -------------------------------------------------------
    # 6. 预测与评估
    # -------------------------------------------------------
    print("\n--- [步骤 6] 目标域预测 ---")
    if len(F_tgt_scaled) > 0:
        # 分批预测防止内存溢出 (虽一般不需要，但保险)
        y_pred_tgt = svm_global.predict(F_tgt_scaled)
        
        # 评估 (如果有真实标签)
        # 注意：这里假设 y_tgt 是有效的，如果全是占位符 0，准确率无意义
        unique_tgt_labels = np.unique(y_tgt)
        if len(unique_tgt_labels) > 1: # 只有当目标域包含至少两种标签时才评估
            print("  - 正在评估目标域准确率...")
            print(classification_report(y_tgt, y_pred_tgt, zero_division=0))
        else:
            print("  - 目标域无有效标签 (或仅有一种标签)，跳过 Accuracy 评估。")
    else:
        print("❌ 警告：目标域特征为空。")
        return

    # -------------------------------------------------------
    # 7. 结果保存 (回溯)
    # -------------------------------------------------------
    print("\n--- [步骤 7] 保存结果 ---")
    full_predictions = pd.Series(data=np.nan, index=df_tgt.index, dtype=float)
    full_predictions.loc[TGT_INDICES] = y_pred_tgt
    
    df_tgt['predicted_multipath'] = full_predictions
    
    OUTPUT_DIR = os.path.join(config.MODEL_SAVE_DIR, "results_single_svm")
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    OUTPUT_PATH = os.path.join(OUTPUT_DIR, "tgt_data_with_single_svm_predictions.csv")
    
    df_tgt.to_csv(OUTPUT_PATH, index=False)
    print(f"✅ 结果已保存至: {OUTPUT_PATH}")

if __name__ == '__main__':
    main_single_svm()

--- [SVM Main] 正在使用设备: cpu ---

--- [步骤 1] 加载原始数据 ---
[load_data] 正在加载目标域数据: 0
[load_data] 目标域加载完成。形状: (12614, 14)
[load_data] 注：源域数据将在 'create_all_sequences' 中逐个加载以保证时序独立性。

--- [步骤 2] 制作序列 ---

--- [序列制作] 正在创建 SVM 所需的序列 (逐个Case处理)... ---
  - 处理源域 Case 1: Case1_Urban_10Hz_with_sat_type.csv
  - X 形状: (91575, 10, 5), y 形状: (91575,), S 形状: (91575,)
  - 处理源域 Case 2: Case1_Suburban_10Hz_with_sat_type.csv
  - X 形状: (91575, 10, 5), y 形状: (91575,), S 形状: (91575,)
  - 处理源域 Case 3: Case2_Urban_10Hz_with_sat_type.csv
  - X 形状: (79712, 10, 5), y 形状: (79712,), S 形状: (79712,)
  - 处理源域 Case 4: Case2_Suburban_10Hz_with_sat_type.csv
  - X 形状: (79750, 10, 5), y 形状: (79750,), S 形状: (79750,)
  - 处理源域 Case 5: Case3_Urban_10Hz_with_sat_type.csv
  - X 形状: (91574, 10, 5), y 形状: (91574,), S 形状: (91574,)
  - 处理源域 Case 6: Case3_Suburban_10Hz_with_sat_type.csv
  - X 形状: (91575, 10, 5), y 形状: (91575,), S 形状: (91575,)
  - 处理源域 Case 7: Case4_Urban_10Hz_with_sat_type.csv
  - X 形状: (91575, 10, 5), y 形状: (91575,), S 形

  - 提取特征中: 100%|██████████| 11077/11077 [01:12<00:00, 152.24it/s]


  - 特征提取完毕。输出形状: (708911, 128)
【✅ 正常】原始特征里没有 NaN。问题出在标准化步骤。
  - 正在执行特征提取...


  - 提取特征中: 100%|██████████| 195/195 [00:01<00:00, 114.43it/s]


  - 特征提取完毕。输出形状: (12443, 128)
  - 源域特征形状: (708911, 128)

--- [步骤 4] 特征标准化 ---
  - 执行特征标准化 (Z-Score)...
  - [调试] 特征均值前5位: [0.59341562 0.77596106 0.33016311 0.36866778 0.40965743]
  - [调试] 特征方差前5位: [1.33573844 1.98276945 0.19791398 0.26715629 0.17905417]
【⚠️ 清理】发现 1240320 NaN / 0 Inf。正在修复...

--- [步骤 5] 数据平衡与 SVM 训练 ---
  - [平衡前] 总体分布: Counter({np.int64(1): 376006, np.int64(0): 332905})
  - 组别统计: GNSS好:102694, GNSS坏:342768, PL好:230211, PL坏:33238
  - 平衡基准数 (min_size): 33238
  - 数据量 132952 > 20000，进行二次随机采样...
  - 最终 SVM 训练集大小: (20000, 128)
  - 开始训练 SVM (RBF Kernel)...
✅ SVM 训练完成。

--- [步骤 6] 目标域预测 ---
  - 目标域无有效标签 (或仅有一种标签)，跳过 Accuracy 评估。

--- [步骤 7] 保存结果 ---
✅ 结果已保存至: c:\Users\yangj\Desktop\GNSS-main\GNSS-main\Transfer Learning\SVM\checkpoints\results_single_svm\tgt_data_with_single_svm_predictions.csv
